# Byte-at-a-time ECB decryption (Harder)
Take your oracle function from #12. Now generate a random count of random bytes and prepend this string to every plaintext. You are now doing:

```
AES-128-ECB(random-prefix || attacker-controlled || target-bytes, random-key)
```

Same goal: decrypt the target-bytes.

### Stop and think for a second.
> What's harder than challenge #12 about doing this? How would you overcome that obstacle? The hint is: you're using all the tools you already have; no crazy math is required.
>
> Think "STIMULUS" and "RESPONSE".


In [1]:
from importlib import reload
import cryptopals
reload(cryptopals)
from cryptopals import *

In [15]:
def prepending_ECB_oracle(plaintext, key):
    '''Oracle that appends a mystery string to the plaintext input before encrypting via ECB.'''
    unknown_string = base64_to_bytes('Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK')
    random_prefix_len = np.random.randint(AES.block_size*10)+1
    random_prefix = generate_random_bytes(random_prefix_len)
    plaintext = pad_pkcs7(random_prefix + to_bytes(plaintext) + unknown_string)
    
    return AES.new(key, AES.MODE_ECB).encrypt(plaintext)

def find_unknown_string_from_prepending_oracle():
    # generate consistent but unknown key
    consistent_unknown_key = generate_random_bytes()
    
    # find unknown message, one byte at a time
    hidden_string = ''
    
    while True:
        idx_next_letter = len(hidden_string)
        idx_start_of_block = idx_next_letter - idx_next_letter%BLOCK_SIZE
        short_pt = chr(0)*(BLOCK_SIZE-1-idx_next_letter%BLOCK_SIZE)

        ct_block_to_char_map = {}
        for i in range(256):
            pt = short_pt + hidden_string + chr(i)
            ct = appending_ECB_oracle(pt, consistent_unknown_key)
            unknown_block = ct[idx_start_of_block:idx_start_of_block+BLOCK_SIZE]
            ct_block_to_char_map[unknown_block] = chr(i)

        ct = appending_ECB_oracle(short_pt, consistent_unknown_key)
        unknown_block = ct[idx_start_of_block:idx_start_of_block+BLOCK_SIZE]
        if unknown_block not in ct_block_to_char_map:
            # signals end of message because padding bytes don't match any possible
            return hidden_string[:-1]
        hidden_string += ct_block_to_char_map[unknown_block]

In [7]:
consistent_unknown_key = generate_random_bytes()

In [23]:
len(prepending_ECB_oracle('hi', consistent_unknown_key))

240